# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [38]:
import pickle
from sklearn.model_selection import train_test_split
import math
import numpy as np

## Load the Data

Start by importing the data from the pickle file.

In [39]:
import cv2
import glob
images_path = '/home/calebkirksey/Code/styx/ros/src/tl_detector/training_images/'
file_extension = '*.jpeg'
light_images = {}
colors = ['red', 'yellow', 'green']
for color in colors:
    image_list = []
    for filename in glob.glob(images_path + color + '/' + file_extension):
        im=cv2.imread(filename)
        image_list.append(im)
    light_images[color] = image_list

In [44]:
features = []
labels = []

image_counts_per_color = []
for color_index in range(len(colors)):
    image_counts_per_color.append(len(light_images[colors[color_index]]))
max_image_count = max(image_counts_per_color)
    
for color_index in range(len(colors)):
    current_color_image_count = image_counts_per_color[color_index]
    for i in range(max_image_count):
        image = light_images[colors[color_index]][i%current_color_image_count]
        features.append(image)
        labels.append(color_index)
        
print(len(labels))
print(len(features))
print(labels)

528
528
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [45]:
data = {}
data['labels'] = np.array(labels)
data['features'] = np.array(features)
data.keys()
label_count = 3

In [46]:
print data['features'].shape[1:]

(40, 150, 3)


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [47]:
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)

In [48]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (40,150,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (40,150,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [49]:
X_train.shape

(396, 40, 150, 3)

In [50]:
type(X_train)

numpy.ndarray

In [51]:
X_train[1]

array([[[185, 145, 117],
        [185, 145, 117],
        [185, 145, 117],
        ..., 
        [169, 145, 127],
        [172, 151, 136],
        [ 61,  44,  31]],

       [[186, 146, 118],
        [186, 146, 118],
        [185, 145, 117],
        ..., 
        [171, 148, 132],
        [ 65,  48,  35],
        [ 52,  36,  23]],

       [[186, 146, 118],
        [186, 146, 118],
        [186, 146, 118],
        ..., 
        [ 60,  41,  26],
        [ 74,  60,  48],
        [ 52,  43,  30]],

       ..., 
       [[ 51,  59,  52],
        [ 29,  37,  30],
        [ 39,  49,  43],
        ..., 
        [ 35,  41,  36],
        [ 35,  41,  36],
        [ 35,  41,  36]],

       [[ 34,  41,  38],
        [ 38,  45,  42],
        [ 37,  44,  41],
        ..., 
        [ 35,  41,  36],
        [ 35,  41,  36],
        [ 35,  41,  36]],

       [[ 35,  41,  40],
        [ 36,  42,  41],
        [ 34,  40,  39],
        ..., 
        [ 35,  41,  36],
        [ 35,  41,  36],
        [ 35,  41,

In [52]:
X_train[1][0]

array([[185, 145, 117],
       [185, 145, 117],
       [185, 145, 117],
       [185, 145, 117],
       [185, 145, 116],
       [185, 145, 116],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [183, 145, 115],
       [183, 145, 115],
       [184, 145, 113],
       [184, 145, 113],
       [184, 145, 113],
       [184, 145, 113],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 116],
       [184, 144, 116],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144, 115],
       [184, 144

In [53]:
X_train[1][0][0]

array([185, 145, 117], dtype=uint8)

In [54]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)

In [17]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

AttributeError: 'module' object has no attribute 'isclose'

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [55]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Convolution2D

In [56]:
# model = Sequential()
# model.add(Dense(128, input_shape=(40*150*3,)))
# model.add(Activation('relu'))
# model.add(Dense(label_count))
# model.add(Activation('softmax'))

IMAGE_WIDTH = 40
IMAGE_HEIGHT = 150
NUM_CLASSES = 3

model = Sequential([
  Convolution2D(16, 3, 3, border_mode='same', subsample=(2, 2), input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES), activation='relu'),
  MaxPooling2D(pool_size=(3, 3)),
  Dropout(0.2),

  Convolution2D(32, 3, 3, border_mode='same', activation='relu'),
  MaxPooling2D(pool_size=(3, 3)),
  Dropout(0.2),

  Convolution2D(64, 3, 3, border_mode='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),

  Flatten(),
  Dense(128, activation='tanh'),
  Dropout(0.3),
  Dense(NUM_CLASSES, activation='softmax'),
])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=(40, 150, ...)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [57]:
# TODO: Compile and train the model here.
from keras.utils import np_utils

# One-hot encode the labels
Y_train = np_utils.to_categorical(y_train, label_count)
Y_val = np_utils.to_categorical(y_val, label_count)

# Reshapen die inputten
X_train_flat = X_train#.reshape(-1, 40*150*3)
X_val_flat = X_val#.reshape(-1, 40*150*3)

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train_flat, Y_train,
                    batch_size=10, nb_epoch=20,
                    verbose=1, validation_data=(X_val_flat, Y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 20, 75, 16)        448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 25, 16)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 6, 25, 16)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 25, 32)         4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 2, 8, 32)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 2, 8, 32)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 8, 64)          18496     
__________

## Save Model

In [58]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


## Load Model

In [59]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


## Run on sample image

In [63]:
red_sample = '/home/calebkirksey/Code/styx/ros/src/tl_detector/training_images/red/1500675686.14_traffic_light.jpeg'
green_sample = '/home/calebkirksey/Code/styx/ros/src/tl_detector/training_images/green/1500489030.95_traffic_light.jpeg'
image = cv2.imread(red_sample)
X = np.array([image])
X = X.astype('float32')
norm = lambda x: x / 255. - 0.5
X = norm(X)

prediction = loaded_model.predict(X, batch_size=1, verbose=0)
print("red: " , prediction[0][0])
print("yellow: " , prediction[0][1])
print("green: " , prediction[0][2])

('red: ', 0.99508768)
('yellow: ', 0.00035904467)
('green: ', 0.0045532738)


## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [45]:
## Reload and re-normalize the data
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [46]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

# TODO: Compile and train the model here.

from keras.layers import Conv2D, Flatten

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)


model = Sequential()
# Note that Conv2D defaults to 'valid' padding
model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 28800)         0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 128)           3686528     flatten_1[0][0]                  
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 43)            5547        dense_7[0][0]                    
Total params: 3692971
_____________________________________________________________________

In [47]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
97.39%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [49]:
## Reload and re-normalize the data
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [50]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

# TODO: Compile and train the model here.
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)


model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_2 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 15, 15, 32)    0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 7200)          0           activation_5[0][0]               
___________________________________________________________________________________________

In [51]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
98.21%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [52]:
## Reload and re-normalize the data
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=42)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
norm = lambda x: x / 255. - 0.5
X_train = norm(X_train)
X_val = norm(X_val)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [53]:
# TODO: Re-construct the network and add dropout after the pooling layer.

# TODO: Compile and train the model here.
from keras.layers import Dropout

model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2,2)))
model.add((Dropout(0.5)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_2[0][0]             
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 15, 15, 32)    0           dropout_1[0][0]                  
___________________________________________________________________________________________

In [54]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**:  
98.25%

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:**  
98.25%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [55]:
# TODO: Load test data
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
# TODO: Preprocess data & one-hot encode the labels
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

# TODO: Evaluate model on test data
model.evaluate(X_test, Y_test)

12608/12630 [============================>.] - ETA: 0s

[0.4935946452191729, 0.91163895483160429]

**Test Accuracy:**
91.16%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.